# Image based recommender system
1. https://github.com/AlexWarembourg/Medium/blob/master/product_recommendaitons.ipynb
2. https://towardsdatascience.com/building-a-similar-images-finder-without-any-training-f69c0db900b5
3. https://www.robertoreif.com/blog/2018/05/14/product-recommendations-using-image-similarity-yy76x

In [1]:
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input

import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from keras.applications import vgg16
                                

Using TensorFlow backend.


In [2]:
import csv
csv_path = "../fashion-product-images-small/styles.csv"

In [14]:
# style_df = pd.read_csv("../fashion-product-images-small/styles.csv")
# style_df.head()

In [3]:
# vgg_model = vgg16.VGG16(weights='imagenet')
# vgg_model.summary()


In [15]:
# now we have vgg model to extract the features of a product image so we can flatten the output and pass it 
# through cosine similarity

In [3]:
# grab the all images
files = [x for x in os.listdir("../fashion-product-images-small/images") if x.endswith('jpg')]

In [4]:
# get random 2000
files = np.random.choice(files, 2000)

In [5]:
files

array(['46093.jpg', '14511.jpg', '7133.jpg', ..., '10389.jpg',
       '50827.jpg', '43598.jpg'], dtype='<U9')

In [23]:
# class Recommender(object):
#     def __init__(self, model, list_of_image, filespath):
#         """initialize the recommender"""
        
#         self.model = model
#         self.filespath = filespath
#         self.list_of_image = list_of_image
#         #since ouput.shape return object dimension just eval it to get integer
#         self.image_width = eval(str(self.model.layers[0].output.shape[1]))
#         self.image_height = eval(str(self.model.layers[0].output.shape[2]))
#         # remove the last layers in order to get features instead of predictions
#         self.image_features_extractor = Model(inputs=self.model.input, 
#                                               outputs=self.model.layers[-2].output)
#         self.pic_to_vector = self_pic_to_matrix()
#         self.sim_table = self.get_similarity(self.pic_to_vector)
    
    
#     def pic_to_matrix():
#         # from vgg 

In [6]:
class ImageRecommender : 
    
    def __init__(self, model, list_of_image, filespath) : 
        self.model = model
        self.filespath = filespath
        self.list_of_image = list_of_image
        #since ouput.shape return object dimension just eval it to get integer ...
        self.image_width = eval(str(self.model.layers[0].output.shape[1]))
        self.image_height = eval(str(self.model.layers[0].output.shape[2]))
        # remove the last layers in order to get features instead of predictions
        self.image_features_extractor = Model(inputs=self.model.input, 
                                              outputs=self.model.layers[-2].output)
        self.processed_image = self.pics_to_array()
        self.sim_table = self.get_similarity(self.processed_image)
        
    def ddl_images(self, image_url) :
        try : 
            return load_img(self.filespath + image_url, 
                            target_size=(self.image_width, self.image_height))
        except OSError : 
            # image unreadable // remove from list
            self.list_of_image = [x for x in self.list_of_image if x != image_url]
            #self.list_of_image.remove(image_url)
            pass
        
    def pics_to_array(self) :
        """
        # convert the PIL image to a numpy array
        # in PIL - image is in (width, height, channel)
        # in Numpy - image is in (height, width, channel)
        # convert the image / images into batch format
        # expand_dims will add an extra dimension to the data at a particular axis
        # we want the input matrix to the network to be of the form (batchsize, height, width, channels)
        # thus we add the extra dimension to the axis 0.
        """
        #from keras.preprocessing.image import load_img,img_to_array
        list_of_expanded_array = list()
        for i in tqdm(range(len(self.list_of_image) - 1)) :
            try :
                tmp = img_to_array(self.ddl_images(self.list_of_image[i]))
                expand = np.expand_dims(tmp, axis = 0)
                list_of_expanded_array.append(expand)
            except ValueError : 
                self.list_of_image = [x for x in self.list_of_image if x != self.list_of_image[i]]
                #self.list_of_image.remove(self.list_of_image[i])
        images = np.vstack(list_of_expanded_array)
        """
        list_of_expanded_array = [try np.expand_dims(img_to_array(self.ddl_images(self.list_of_image[i])), axis = 0) except ValueError pass \
                                  for i in tqdm(range(len(self.list_of_image)))]
        images = np.vstack(list_of_expanded_array)
        #from keras.applications.imagenet_utils import preprocess_input()
        # prepare the image for the  model"
        """
        return preprocess_input(images)
    
    def get_similarity(self, processed_imgs) :
        print('============ algorithm predict featurs =========')
        imgs_features = self.image_features_extractor.predict(processed_imgs)
        print("Our image has %i features:" %imgs_features.size)
        cosSimilarities = cosine_similarity(imgs_features)
        cos_similarities_df = pd.DataFrame(cosSimilarities, 
                                           columns=self.list_of_image[:len(self.list_of_image) -1],
                                           index=self.list_of_image[:len(self.list_of_image) -1])
        return cos_similarities_df
    
    def most_similar_to(self, given_img, nb_closest_images = 5):

        print("-----------------------------------------------------------------------")
        print("original manga:")

        original = self.ddl_images(given_img)
        plt.imshow(original)
        plt.show()

        print("-----------------------------------------------------------------------")
        print("most similar manga:")

        closest_imgs = self.sim_table[given_img].sort_values(ascending=False)[1:nb_closest_images+1].index
        closest_imgs_scores = self.sim_table[given_img].sort_values(ascending=False)[1:nb_closest_images+1]

        for i in range(0,len(closest_imgs)):
            original = self.ddl_images(closest_imgs[i])
            plt.imshow(original)
            plt.show()
            print("similarity score : ",closest_imgs_scores[i])

In [ ]:
%matplotlib inline 
from tqdm import tqdm

vgg_model = vgg16.VGG16(weights='imagenet')
# vgg_model.summary()
print("Start")

list_of_pretrained = [vgg_model]
filespath = "../fashion-product-images-small/images/"
for pretrained_model in list_of_pretrained : 
    print('=========================================')
    print(' /t pretrained model %s are running' %pretrained_model)
    print('=========================================')
    pretrained_recommender = ImageRecommender(pretrained_model, files, filespath)
   
    print('=========================================')
    pretrained_recommender.most_similar_to(files[777])
    print('=========================================')
    pretrained_recommender.most_similar_to(files[23])
    print('\n')
    
print("done")

  2%|▏         | 36/1999 [00:00<00:05, 355.67it/s]

Start
 /t pretrained model <keras.engine.training.Model object at 0x10d8b36a0> are running


100%|██████████| 1999/1999 [00:04<00:00, 408.94it/s]


============ algorithm predict featurs =========
